In [1]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
from datetime import date, timedelta
import io
import time
import sqlite3
import pandas as pd
import csv

In [3]:
df = pd.read_csv('news.csv')
df

,date,press,title,url
0,20230101,32,“아내와 함께 떠난 딸…마지막이 될 줄 몰랐다”,https://n.news.naver.com/article/032/000319644...
1,20230101,32,"뜸 들이다 끝난 ‘더 글로리’···복수한다며, 남자가 왜 거기서 나와[리뷰]",https://n.news.naver.com/article/032/000319637...
2,20230101,32,이것은 발 토시가 아니다…세련미 더해 돌아온 ‘라떼는’ 패션,https://n.news.naver.com/article/032/000319630...
3,20230101,32,"경찰, ‘방음터널 화재’ 폐기물 운반용 집게 트럭 소유 업체 압수수색",https://n.news.naver.com/article/032/000319636...
4,20230101,32,다시 만난 ‘자낳세’…“월급만으로 살 수 없단 생각엔 변함 없어”,https://n.news.naver.com/article/032/000319644...
...,...,...,...,...
63975,20231116,469,"""한동훈 탄핵 필요하면 검토""... '검사 탄핵' 물러서지 않는 민주당",https://n.news.naver.com/article/469/000077058...
63976,20231116,469,제2의 손흥민·이강인도 '전국대회 8강' 못 들면 대학 못 간다?,https://n.news.naver.com/article/469/000077038...
63977,20231116,469,尹대통령에 '쪽지메모' 전달한 정대철 “검사출신 넘어 폭넓은 인사 등용하라”[정치행간],https://n.news.naver.com/article/469/000077045...
63978,20231116,469,"[르포]""MMORPG 한계 왔다""...택진이형까지 새 장르·플랫폼으로 위기 탈출 나섰다",https://n.news.naver.com/article/469/000077061...


In [8]:
df['url'][0]

'https://n.news.naver.com/article/032/0003196443?ntype=RANKING'

In [11]:
# url 하나하나 접근해 뉴스 본문 수집

def news_content_crawling(data):
    headers = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    f = open('news_content.csv', 'w', newline='', encoding='utf-8')
    wtr = csv.writer(f)
    wtr.writerow(["title", "content"])
    
    for i in tqdm(range(len(data["url"]))):
        if i % (len(data['url']) // 20) == 100:
            time.sleep(60)
        else:
            time.sleep(1)
        
        url = data['url'][i]
        try:
            rs = requests.get(url, headers=headers)
        except:
            print(f'{i} : RETRY-TIME ERROR!')
            time.sleep(5)
            rs = requests.get(url, headers=headers)
            
        if rs.status_code != 200:
            print('CONTENT REQUEST FAILED! : ', url)
            return "REQUEST_ERROR"
        
        rs.encoding = 'UTF-8' # html의 인코딩 방식 확인하기
        soup = BeautifulSoup(rs.text, 'html.parser')
        title = soup.select_one('#title_area > span').get_text()
        raw_txt = soup.select_one('#newsct_article')
        try:
            for i in raw_txt.find_all('span'):
                i.decompose()
            raw_txt = raw_txt.get_text(separator='\n')
            #e-mail/기자 제거
            pt = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
            raw_txt = re.sub(pattern=pt, repl='', string=raw_txt)
            raw_txt = raw_txt.replace('기자', '')
            raw_txt = raw_txt.strip()
        
            wtr.writerow([title, raw_txt])

        except:
            print('CONTENT CRAWL FAILED! : ', url)
            return "CONTENT_CRAWL_ERROR"

In [12]:
news_content_crawling(df)

  0%|          | 0/63980 [00:00<?, ?it/s]

  0%|          | 24/63980 [00:46<34:19:52,  1.93s/it]


KeyboardInterrupt: 

## 멀티 스레드 크롤링

In [17]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
import time
import re
from concurrent.futures import ThreadPoolExecutor

def crawl_url(url):
    headers = {'user-agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"}
    
    
    try:
        time.sleep(0.02)
        rs = requests.get(url, headers=headers)
    except:
        print(f'{url} : RETRY-TIME ERROR!')
        time.sleep(5)
        rs = requests.get(url, headers=headers)

    if rs.status_code != 200:
        print('CONTENT REQUEST FAILED! : ', url)
        return None

    try:
        rs.encoding = 'UTF-8'
        soup = BeautifulSoup(rs.text, 'html.parser')
        title = soup.select_one('#title_area > span').get_text()
        raw_txt = soup.select_one('#newsct_article')
    except Exception as e:
        print(e)
        return None

    try:
        for i in raw_txt.find_all('span'):
            i.decompose()
        raw_txt = raw_txt.get_text(separator='\n')
        pt = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
        raw_txt = re.sub(pattern=pt, repl='', string=raw_txt)
        raw_txt = raw_txt.replace('기자', '')
        raw_txt = raw_txt.strip()
        return title, raw_txt
    except:
        print('CONTENT CRAWL FAILED! : ', url)
        return None

def news_content_crawling(data):
    f = open('news_content.csv', 'w', newline='', encoding='utf-8')
    wtr = csv.writer(f)
    wtr.writerow(["title", "content"])
    urls = data['url'].tolist()

    def process_url(url):
        result = crawl_url(url)
        if result:
            wtr.writerow(result)

    with ThreadPoolExecutor(max_workers=15) as executor:
        list(tqdm(executor.map(process_url, urls), total=len(urls)))

    f.close()

# 크롤링 실행
news_content_crawling(df)

  2%|▏         | 992/63980 [02:02<3:14:20,  5.40it/s]

CONTENT REQUEST FAILED! :  https://n.news.naver.com/article/469/0000716671?ntype=RANKING


  4%|▍         | 2714/63980 [05:52<1:48:41,  9.39it/s]

CONTENT REQUEST FAILED! :  https://n.news.naver.com/article/022/0003774045?ntype=RANKING


  4%|▍         | 2763/63980 [05:59<2:54:45,  5.84it/s]

...

CONTENT REQUEST FAILED! :  https://n.news.naver.com/article/025/0003313158?ntype=RANKING


 89%|████████▉ | 56819/63980 [2:32:43<15:28,  7.72it/s]

CONTENT REQUEST FAILED! :  https://n.news.naver.com/article/020/0003525064?ntype=RANKING


 89%|████████▉ | 57245/63980 [2:33:56<18:06,  6.20it/s]


PermissionError: [Errno 13] Permission denied

In [16]:
df.loc[7510]['url']

'https://n.news.naver.com/article/022/0003781245?ntype=RANKING'

In [32]:
import chardet

with open('news_content.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [50]:
pd.read_csv("news_content.csv", encoding='cp949', on_bad_lines='skip')

UnicodeDecodeError: 'cp949' codec can't decode byte 0x91 in position 17: illegal multibyte sequence

In [51]:
with open('news_content.csv', encoding='utf-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for c in reader:
        for k, v in c.items():
            print(k,v)
    

title 중 감염병 유명 전문가 “상하이 코로나 감염자 1000만명 추산”
content 중국의 급속한 코로나19 확산세 속에 경제 수도 상하이의 코로나19 감염자 수가 1000만 명이 넘는 것으로 보인다고 현지의 유명 감염병 전문가가 추산했다.
31일 중국 공산당 기관지 인민일보에 따르면 장원훙 상하이 푸단대 부속 화산병원 주임은 이 매체의 소셜미디어 계정인 ‘다장둥’과의 화상 인터뷰에서 “상하이는 지난 22일부터 코로나19 감염자 정점 상태에 도달했고, 추산에 의하면 현재 감염자 수는 1000만명 이상”이라고 말했다.
이어 “1000만명의 감염자 중 0.5%가 병원에 입원했다고 가정하면 최근 몇 주 동안 5만명이 병원에 입원했다는 의미”라며 “대형병원마다 수천명을 치료하고 있다”고 설명했다.
또 “감염자 중 중증환자의 비율이 낮지만, 감염자 수가 절대적으로 많다”며 “이것이 오미크론 감염의 특징으로, 우리는 모든 나라가 코로나 개방 초기에 겪었던 엄중한 도전에 직면해 있다”고 밝혔다.
장 주임은 보건소부터 대형병원까지 상하이 모든 의료시설의 의사와 간호사가 코로나19 감염자 치료에 투입됐으며 중증환자 치료법을 개선하기 위해 총력을 기울이고 있다고 말했다.
그는 “치료 시설과 의료 인력 확충, 산소요법과 항바이러스제 제공, 규범화된 진료수첩 제공 등 사회 전반의 치료법을 개선하고, 많은 환자가 발병 72시간 이내에 치료받을 수 있도록 해야 한다”며 “병상 수를 확충하고 응급환자에 대해서는 24시간 이내에 치료받을 수 있도록 해야 한다”고 강조했다.
title 이것은 발 토시가 아니다…세련미 더해 돌아온 ‘라떼는’ 패션
content 그해의 유행을 속성으로 배울 수 있는 시간, 바로 연말 시상식 무대다. 지난 16일 <2022 KBS 가요 대축제>에서 걸그룹 ITZY의 멤버 유나가 선보인 이효리의 ‘유고걸(U-Go-Girl)’ 커버 무대가 그랬다.
화려한 퍼포먼스만큼이나 주목받은 의상은 ‘크롭트 셔츠’와 ‘로 라이즈 진’이다. ‘배꼽티’와 ‘골반 바지’라는 

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xae in position 5732: invalid start byte